In [1]:
from plot import *
from filter import *
from analysis import *

In [2]:
# Files naming
graph_file = 'shinkansen_graph.pkl'
graph_file_filtered = 'shinkansen_graph_filtered.pkl'
# Pre-processed data
shinkansen_join = 'data/shinkansen_join.csv'
# Output images
output_image = 'shinkansen_lines_visualization.png'
output_image_filtered = 'shinkansen_lines_visualization_filtered.png'
# Filter settings
min_degree = 2
top_n = 100

In [3]:
try:
    G = load_graph(graph_file)
    print('Graph loaded from file')
except:
    print('Graph file not found, creating graph')
    G = create_graph_form_csv(shinkansen_join)
    save_graph(G, graph_file)
    print('Graph saved to file')

Graph loaded from file


In [4]:
nodes_num = G.number_of_nodes()
edges_num = G.number_of_edges()
print(f'Number of nodes: {nodes_num}')
print(f'Number of edges: {edges_num}')

Number of nodes: 124
Number of edges: 122


In [8]:
visualize_graph(G, output_image, size=15)
print(f'Graph visualization saved to {output_image}')

Graph visualization saved to shinkansen_lines_visualization.png


In [6]:
print(find_degree(G, '東京'))
print(find_degree(G, '新大阪'))

2
2


In [7]:
print(find_all_path(G, '東京', '新大阪'))
print(find_shortest_path(G, '東京', '新大阪'))

1
['東京', '品川', '新横浜', '小田原', '熱海', '三島', '新富士', '静岡', '掛川', '浜松', '豊橋', '三河安城', '名古屋', '岐阜羽島', '米原', '京都', '新大阪']


In [10]:
json_str = dict_to_json(find_all_communities(G))
with open('communities.json', 'w') as f:
    f.write(json_str)

In [11]:
print(find_smallest_community(G))
print(find_largest_community(G))

(9, ['武雄温泉', '嬉野温泉', '新大村', '諫早', '長崎'])
(8, ['小倉', '博多', '新鳥栖', '久留米', '筑後船小屋', '新大牟田', '新玉名', '熊本', '新八代', '新水俣', '出水', '川内', '鹿児島中央'])
